# Model Deployment

In [1]:
import numpy as np 
np.set_printoptions(suppress=True)

import pandas as pd 
import json
from flask import request, jsonify
from flask import Flask, Blueprint
from flask import render_template

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
hello = Blueprint('hello', __name__)

@hello.route('/')
def index():
    #return 'Flask default function - logreg'
    return render_template("index.html")

@hello.route('/hi')
def goodbye():
    return 'Hey Hello World!'

### Load the Model

In [3]:
model = pickle.load(open('logreg.pickle', 'rb' ))

### Load the Scaler from pickle file

In [4]:
with open('scaler.pickle', 'rb') as f2:
    scaler_pkl = pickle.load(f2)    

### Load the Encoder from pickle file

In [5]:
with open('encoder.pickle', 'rb') as f3:
    encoder_pkl = pickle.load(f3)


### Process input data

In [6]:
# Function to convert an array to a dataframe
def convert_to_df(arr):
    column_names = ['Administrative', 'ProductRelated_Duration', 'BounceRates', 'PageValues', 'SpecialDay', 'Month']
    # Create a DataFrame
    df = pd.DataFrame([arr], columns=column_names)
    df = df.astype(float)

    return df

#This function encodes the 'SpecialDay', 'Month' in the dataframe using the pickled encoder and returns the modified dataframe
def one_hot_encoding(df):
    encoded_data = encoder_pkl.transform(df[['SpecialDay', 'Month']])
    # Convert the encoded data to a DataFrame with meaningful column names
    encoded_df = pd.DataFrame(
        encoded_data,
        columns=[f"{col}_{val}" for col, vals in zip(['SpecialDay', 'Month'], encoder_pkl.categories_) for val in vals]
    )

    # Concatenate the original DataFrame with the encoded DataFrame
    encoded_data2 = pd.concat([df, encoded_df], axis=1)

    # Drop the original columns that were encoded
    encoded_data2 = encoded_data2.drop(['SpecialDay', 'Month'], axis=1)
    return encoded_data2

#This function scales the data in a dataframe using the pickled scaler
def scale_input_data(df):
    input_scaled = scaler_pkl.transform(df)
    return input_scaled

### Endpoint to serve incoming requests

In [7]:
@hello.route('/api', methods=['GET'])
def api():
    field1 = request.args.get('field1')
    field2 = request.args.get('field2')
    field3 = request.args.get('field3')
    field4 = request.args.get('field4')
    field5 = request.args.get('field5')
    field6 = request.args.get('field6')
    
    # For example, print data to the console
    
    input_arr=[field1, field2, field3, field4, field5, field6]
    print(f"Input data: {input_arr}")
    print(" ")
    
    input_df = convert_to_df(input_arr)
    one_hot_encoded_df = one_hot_encoding(input_df)
    
    ##----only for printing purpose-----------------------
    array_as_list = one_hot_encoded_df.values.flatten().tolist()
    formatted_string = ', '.join(map(str, array_as_list))
    print(f"Encoded data: [{formatted_string}]")
    print(" ")
    #-----------------------------------------------------
    
    input_scaled=scale_input_data(one_hot_encoded_df)
    
    ##----only for printing purpose-----------------------
    array_as_list2 = input_scaled.flatten().tolist()
    rounded_list = [round(element, 2) for element in array_as_list2]
    formatted_string2 = ', '.join(map(str, rounded_list))
    print(f"Scaled data: [{formatted_string2}]")
    print(" ")
    #-----------------------------------------------------
    
    
    y_pred = model.predict(input_scaled)
    #print(y_pred[0])

    y_prob = model.predict_proba(input_scaled)
    y_prob = np.round(y_prob, 3)
    #print(y_prob[0])

    # Create a dictionary
    result_dict = {
        "predicted_class": y_pred.tolist(),
        "probabilities": y_prob.tolist()
    }
    
    print(f"Model Inference Results: [{result_dict}]")
    # Convert the dictionary to JSON
    json_data = json.dumps(result_dict)

    return json_data

### Start Application

In [8]:
if __name__ == '__main__':
    app = Flask(__name__)
    app.register_blueprint(hello, url_prefix='/')

    app.run(host='0.0.0.0', port=5500)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5500
 * Running on http://192.168.1.5:5500
Press CTRL+C to quit
192.168.1.5 - - [05/Feb/2024 22:37:47] "GET / HTTP/1.1" 200 -
192.168.1.5 - - [05/Feb/2024 22:37:48] "GET /favicon.ico HTTP/1.1" 404 -
